In [ ]:
!pip install pandas nltk
!pip install fastparquet
!pip install hdfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34324 sha256=3da3e9d6f255e0c3ea0d1b5ed29d9de76a03eb5f37b891c4d8407922783131d6
  Stored in directory: /root/.cache/pip/wheels/b9/1d/dc/eb0833be25464c359903d356c4204721c6a672c26ff164cdc3
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7667068d6a7b10224b1d762f46b1885adfe3d5baa7d206e7a337f58978c979ca
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built hdfs docopt


In [4]:
from zipfile import ZipFile

zip_paths = ["/content/archive (1).zip", "/content/archive.zip"]
extract_paths = ["/content", "/content"]

for zip_path, extract_path in zip(zip_paths, extract_paths):
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

In [13]:
import pandas as pd
import nltk
nltk.download('punkt_tab')
from nltk.stem import WordNetLemmatizer
import string

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
road = pd.read_csv('/content/Road Accident Data.csv')
world = pd.read_csv('/content/World Important Dates.csv')

In [7]:
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
def normalize_text(text):
    if pd.isna(text):
        return text
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = nltk.word_tokenize(text)
    text = ' '.join(lemmatizer.lemmatize(word) for word in words)
    return text

In [14]:
for col in road.select_dtypes(exclude=['number']).columns:
    road[col] = road[col].apply(normalize_text)

for col in world.select_dtypes(exclude=['number']).columns:
    world[col] = world[col].apply(normalize_text)

In [15]:
road.head()

,Accident_Index,Accident Date,Day_of_Week,Junction_Control,Junction_Detail,Accident_Severity,Latitude,Light_Conditions,Local_Authority_(District),Carriageway_Hazards,...,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type
0,200901bs70001,112021,thursday,give way or uncontrolled,t or staggered junction,serious,51.512273,daylight,kensington and chelsea,NaN,...,1,2,metropolitan police,dry,one way street,30,1511,urban,fine no high wind,car
1,200901bs70002,152021,monday,give way or uncontrolled,crossroad,serious,51.514399,daylight,kensington and chelsea,NaN,...,11,2,metropolitan police,wet or damp,single carriageway,30,1059,urban,fine no high wind,taxiprivate hire car
2,200901bs70003,142021,sunday,give way or uncontrolled,t or staggered junction,slight,51.486668,daylight,kensington and chelsea,NaN,...,1,2,metropolitan police,dry,single carriageway,30,1419,urban,fine no high wind,taxiprivate hire car
3,200901bs70004,152021,monday,auto traffic signal,t or staggered junction,serious,51.507804,daylight,kensington and chelsea,NaN,...,1,2,metropolitan police,frost or ice,single carriageway,30,810,urban,other,motorcycle over 500cc
4,200901bs70005,162021,tuesday,auto traffic signal,crossroad,serious,51.482076,darkness light lit,kensington and chelsea,NaN,...,1,2,metropolitan police,dry,single carriageway,30,1725,urban,fine no high wind,car


In [16]:
world.head()

,Sl. No,Name of Incident,Date,Month,Year,Country,Type of Event,Place Name,Impact,Affected Population,Important Person/Group Responsible,Outcome
0,1,indus valley civilization flourish,unknown,unknown,2600 bc,india,civilization,indus valley,development of one of the world earliest urban...,local inhabitant,indus valley people,positive
1,2,battle of the ten king,unknown,unknown,1400 bc,india,battle,punjab,rigvedic tribe consolidated their control over...,rigvedic tribe,sudas,positive
2,6,establishment of the delhi sultanate,unknown,unknown,1206,india,political,delhi,muslim rule established in part of india,people of delhi and surrounding region,qutbunknownudunknowndin aibak,mixed
3,7,battle of panipat,21,april,1526,india,battle,panipat,foundation of the mughal empire in india,northern indian kingdom,babur,mixed
4,8,establishment of british raj,1,may,1858,india,colonial,whole india,start of direct british governance in india,indian subcontinent,british east india companyempire,negative


In [17]:
import fastparquet
road.to_parquet('road.parquet', engine='fastparquet')
world.to_parquet('world.parquet', engine='fastparquet')